In [18]:
import numpy as np

def simplex_revisado(c, A, b, log=True):

    m, n = A.shape
    num_artificiais = 0
    indices_base = list(range(n, n + m))  # Variáveis de folga iniciais na base
    Ab = np.eye(m)  # Ab é Matriz básica inicial, matriz idt
    xb = b.copy()  # xb Solução básica inicial
    cb = np.zeros(m)  # cb Custos básicos iniciais, o custo das variáveis de folga = 0

    log_completo = []
    iteracao = 0

    while True:
        iteracao += 1
        if log:
            log_completo.append(f"\n Iteração {iteracao}")
            log_completo.append(f"Índices da base (B): {[int(i) for i in indices_base]}") # Convertendo para int
            log_completo.append(f"Matriz básica (Ab):\n{Ab}")
            log_completo.append(f"Solução básica (xb): {xb}")
            log_completo.append(f"Custos básicos (cb): {cb}")

        # Faz calculos para os custos reduzidos zj e cj, para variáveis não básicas
        zj_cj = np.zeros(n)
        for j in range(n):
            if j not in indices_base:
                aj = A[:, j]
                zj = cb @ np.linalg.solve(Ab, aj)
                zj_cj[j] = zj - c[j]
                if log:
                    log_completo.append(f"Custo reduzido (z{j+1} - c{j+1}): {zj_cj[j]:.4f}")

        if np.all(zj_cj <= 1e-6):  # Solução ótima. Valor função objetivo. Todos os custos reduzidos são não-positivos
            valor_otimo = cb @ xb
            solucao_otima = np.zeros(n)
            for i in range(m):
                if indices_base[i] < n:
                    solucao_otima[indices_base[i]] = xb[i]
            if log:
                log_completo.append("\nSolução Ótima")
                log_completo.append(f"Valor ótimo da função objetivo: {valor_otimo:.4f}")
                log_completo.append(f"Solução ótima (x): {solucao_otima}")
            return solucao_otima, valor_otimo, log_completo

        # Aqui seleciono a variável para ENTRAR NA BASE, a com o CUSTO REDUZIDO MAIS REDUZIDO.
        j_entra = np.argmax(zj_cj)
        if zj_cj[j_entra] <= 1e-6:
            valor_otimo = cb @ xb
            solucao_otima = np.zeros(n)
            for i in range(m):
                if indices_base[i] < n:
                    solucao_otima[indices_base[i]] = xb[i]
            if log:
                log_completo.append("\nSolução Ótima Encontrada (alternativa)")
                log_completo.append(f"Valor ótimo da função objetivo: {valor_otimo:.4f}")
                log_completo.append(f"Solução ótima (x): {solucao_otima}")
            return solucao_otima, valor_otimo, log_completo

        if log:
            log_completo.append(f"Variável para entrar na base (xj): x{j_entra + 1}")

        # Direção de melhoria (yj)
        aj = A[:, j_entra]
        yj = np.linalg.solve(Ab, aj)
        if log:
            log_completo.append(f"Direção de melhoria (yj):\n{yj}")

        #  Aqui seleciono a variavel para SAIR DA BASE, a RAZÃO MINIMA POSITIVA.
        razoes = np.array([xb[i] / yj[i] if yj[i] > 1e-9 else np.inf for i in range(m)])
        i_sai = np.argmin(razoes)
        #  Atribuindo i_sai para variavel linha_pivo_indice
        linha_pivo_indice = i_sai

        if np.all(razoes <= 0):
            raise ValueError("Problema ilimitado")

        if log:
            log_completo.append(f"Razões (xb_i / yj_i): {razoes}")
            log_completo.append(f"Variável para sair da base (xi): x{indices_base[i_sai] + 1}")

        # Atualizando: a base, a matriz básica, a solução básica e os custos básicos
        coluna_pivo = yj
        linha_pivo_indice = i_sai
        elemento_pivo = coluna_pivo[linha_pivo_indice]

        # Atualizo a matriz básica
        # substituindo a coluna da variável que sai
        # pela coluna da que entra
        Ab[:, linha_pivo_indice] = aj

        # Atualizo a solução básica
        xb_novo = np.zeros(m)
        for i in range(m):
            if i != linha_pivo_indice:
                xb_novo[i] = xb[i] - coluna_pivo[i] / elemento_pivo * xb[linha_pivo_indice]
            else:
                xb_novo[i] = xb[linha_pivo_indice] / elemento_pivo
        xb = xb_novo

        # Atualizo os custos básicos
        cb[linha_pivo_indice] = c[j_entra]

        # Atualizo os índices da base
        indices_base[linha_pivo_indice] = j_entra

# Problema maximização
c = np.array([-50, -60, -45, -70, -55]) # Coeficientes da função objetivo.
A = np.array([
    [2, 1.5, 3, 2.5, 1],
    [1, 1.5, 0.5, 2, 2.5],
    [3, 2.5, 2, 3.5, 2]
]) # Matriz A original. Matriz dos coeficientes das restrições.
b = np.array([400, 300, 200]) #Vetor dos limites das restrições.

# São as restrições: Madeira em m², Tecido em m, e tempo de produção em horas.

# Rodando o simplex e seus logs.
solucao_otima_manual, valor_otimo_manual, log_completo = simplex_revisado(c, A, b, log=True)

# Solução primal
print("\nSolução Primal")
print("Quantidade a produzir de cada produto:")
for i, quantidade in enumerate(solucao_otima_manual):
    print(f"Produto {i+1}: {quantidade:.2f} unidades")
print(f"Lucro máximo total: R$ {-valor_otimo_manual:.2f}")

# 1º Log
print("\n 1º Log")
for linha in log_completo:
    print(linha)

# Variáveis de folga para identificar seus índices
num_restricoes = A.shape[0]
num_variaveis = A.shape[1]
A_com_folgas = np.hstack((A, np.eye(num_restricoes)))
c_com_folgas = np.hstack((c, np.zeros(num_restricoes)))


Solução Primal
Quantidade a produzir de cada produto:
Produto 1: 0.00 unidades
Produto 2: 0.00 unidades
Produto 3: 0.00 unidades
Produto 4: 0.00 unidades
Produto 5: 100.00 unidades
Lucro máximo total: R$ 5500.00

 1º Log

 Iteração 1
Índices da base (B): [5, 6, 7]
Matriz básica (Ab):
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Solução básica (xb): [400 300 200]
Custos básicos (cb): [0. 0. 0.]
Custo reduzido (z1 - c1): 50.0000
Custo reduzido (z2 - c2): 60.0000
Custo reduzido (z3 - c3): 45.0000
Custo reduzido (z4 - c4): 70.0000
Custo reduzido (z5 - c5): 55.0000
Variável para entrar na base (xj): x4
Direção de melhoria (yj):
[2.5 2.  3.5]
Razões (xb_i / yj_i): [160.         150.          57.14285714]
Variável para sair da base (xi): x8

 Iteração 2
Índices da base (B): [5, 6, 3]
Matriz básica (Ab):
[[1.  0.  2.5]
 [0.  1.  2. ]
 [0.  0.  3.5]]
Solução básica (xb): [257.14285714 185.71428571  57.14285714]
Custos básicos (cb): [  0.   0. -70.]
Custo reduzido (z1 - c1): -10.0000
Custo reduzido (z2

In [22]:
def simplex_revisado_dual(c, A, b, log=True):
    # Simplex Revisado até o critério de otimalidade, mesmo código acima.
    m, n = A.shape
    num_artificiais = 0
    indices_base = list(range(n, n + m))
    Ab = np.eye(m)
    xb = b.copy()
    cb = np.zeros(m)

    log_completo = []
    iteracao = 0

    while True:
        iteracao += 1
        if log:
            log_completo.append(f"\nIteração {iteracao} ---")
            log_completo.append(f"Índices da base (B): {[int(i) for i in indices_base]}") # Convertendo para int
            log_completo.append(f"Matriz básica (Ab):\n{Ab}")
            log_completo.append(f"Solução básica (xb): {xb}")
            log_completo.append(f"Custos básicos (cb): {cb}")

        zj_cj_completo = np.zeros(n + m)
        # Custos reduzidos para as variáveis originais
        for j in range(n):
            aj = A[:, j]
            zj = cb @ np.linalg.solve(Ab, aj)
            zj_cj_completo[j] = zj - c[j]
            if log:
                log_completo.append(f"Custo reduzido (z{j+1} - c{j+1}): {zj_cj_completo[j]:.4f}")

        # Custos reduzidos para as variáveis de folga, cj = 0 para folgas
        for j in range(m):
            indice_folga = n + j
            e_j = np.zeros(m)
            e_j[j] = 1
            zj_folga = cb @ np.linalg.solve(Ab, e_j)
            zj_cj_completo[indice_folga] = zj_folga - 0
            if log:
                log_completo.append(f"Custo reduzido (z_s{j+1} - c_s{j+1}): {zj_cj_completo[indice_folga]:.4f}")

        if np.all(zj_cj_completo[:n] <= 1e-6):
            valor_otimo = cb @ xb
            solucao_otima = np.zeros(n)
            for i in range(m):
                if indices_base[i] < n:
                    solucao_otima[indices_base[i]] = xb[i]
            if log:
                log_completo.append("\nSolução Ótima Encontrada")
                log_completo.append(f"Valor ótimo da função objetivo: {valor_otimo:.4f}")
                log_completo.append(f"Solução ótima (x): {solucao_otima}")

            # Extração da solução dual dos custos reduzidos das variáveis de folga
            solucao_dual = zj_cj_completo[n:]
            return solucao_otima, valor_otimo, log_completo, solucao_dual

        # Código de seleção da variável para entrar/sair, mesmo código acima
        j_entra = np.argmax(zj_cj_completo[:n])
        if zj_cj_completo[j_entra] <= 1e-6:
            valor_otimo = cb @ xb
            solucao_otima = np.zeros(n)
            for i in range(m):
                if indices_base[i] < n:
                    solucao_otima[indices_base[i]] = xb[i]
            if log:
                log_completo.append("\nSolução Ótima Encontrada (alternativa)")
                log_completo.append(f"Valor ótimo da função objetivo: {valor_otimo:.4f}")
                log_completo.append(f"Solução ótima (x): {solucao_otima}")
                solucao_dual = zj_cj_completo[n:]
            return solucao_otima, valor_otimo, log_completo, solucao_dual

        if log:
            log_completo.append(f"Variável para entrar na base (xj): x{j_entra + 1}")

        aj = A[:, j_entra]
        yj = np.linalg.solve(Ab, aj)
        if log:
            log_completo.append(f"Direção de melhoria (yj):\n{yj}")

        razoes = np.array([xb[i] / yj[i] if yj[i] > 1e-9 else np.inf for i in range(m)])
        i_sai = np.argmin(razoes)

        if np.all(razoes <= 0):
            raise ValueError("Problema ilimitado")

        if log:
            log_completo.append(f"Razões (xb_i / yj_i): {razoes}")
            log_completo.append(f"Variável para sair da base (xi): x{indices_base[i_sai] + 1}")

        coluna_pivo = yj
        linha_pivo_indice = i_sai
        elemento_pivo = coluna_pivo[linha_pivo_indice] # Atribuindo o elemento pivo

        Ab[:, linha_pivo_indice] = aj
        xb_novo = np.zeros(m)
        for i in range(m):
            if i != linha_pivo_indice:
                xb_novo[i] = xb[i] - coluna_pivo[i] / elemento_pivo * xb[linha_pivo_indice]
            else:
                xb_novo[i] = xb[linha_pivo_indice] / elemento_pivo
        xb = xb_novo
        cb[linha_pivo_indice] = c[j_entra]
        indices_base[linha_pivo_indice] = j_entra

# Rodando para obter a solução dual
solucao_otima_manual, valor_otimo_manual, log_completo, solucao_dual_manual = simplex_revisado_dual(c, A, b, log=True)

# Solução dual
print("\n2º Log")
print("\nSolução Dual")
print("Valores das variáveis duais, preços sombra dos recursos") # valor marginal que se ganha ou se perde ao alterar a disponibilidade de um recurso
print(f"Madeira: R$ {solucao_dual_manual[0]:.2f} por m²")
print(f"Tecido: R$ {solucao_dual_manual[1]:.2f} por m")
print(f"Tempo de Produção: R$ {solucao_dual_manual[2]:.2f} por hora")

print("\nRecursos abundantes e escassos")

# Folgas na solução ótima
folgas_otimas = b - A @ solucao_otima_manual
print("\nFolgas nos recursos na solução ótima:")
print(f"Madeira: {folgas_otimas[0]:.2f} m²")
print(f"Tecido: {folgas_otimas[1]:.2f} m")
print(f"Tempo de Produção: {folgas_otimas[2]:.2f} horas")

recursos_escassos = np.where(folgas_otimas < 1e-6)[0]
if len(recursos_escassos) > 0:
    print("\nRecursos Escassos, igual a zero, são totalmente utilizados:")
    for i in recursos_escassos:
        if i == 0:
            print("Madeira")
        elif i == 1:
            print("Tecido")
        elif i == 2:
            print("Tempo de Produção")
else:
    print("\nNão há recursos totalmente utilizados.")

recursos_abundantes = np.where(folgas_otimas > 1e-6)[0]
if len(recursos_abundantes) > 0:
    print("\nRecursos Abundantes, maior ou igual a zero possui folga:")
    for i in recursos_abundantes:
        if i == 0:
            print(f"Madeira: {folgas_otimas[0]:.2f} m²")
        elif i == 1:
            print(f"Tecido: {folgas_otimas[1]:.2f} m")
        elif i == 2:
            print(f"Tempo de Produção: {folgas_otimas[2]:.2f} horas")
else:
    print("\nNão há recursos com folga.")


2º Log

Solução Dual
Valores das variáveis duais, preços sombra dos recursos
Madeira: R$ 0.00 por m²
Tecido: R$ 0.00 por m
Tempo de Produção: R$ -27.50 por hora

Recursos abundantes e escassos

Folgas nos recursos na solução ótima:
Madeira: 300.00 m²
Tecido: 50.00 m
Tempo de Produção: -0.00 horas

Recursos Escassos, igual a zero, são totalmente utilizados:
Tempo de Produção

Recursos Abundantes, maior ou igual a zero possui folga:
Madeira: 300.00 m²
Tecido: 50.00 m
